# 1-D Convonets

## Introduction

- 1-D convonents can be used for text data.
- They are faster than RNNs and at times more useful for audio / signal processing.

- Such 1D convolution layers can recognize local patterns in a sequence. Because the
same input transformation is performed on every patch, a pattern learned at a certain
position in a sentence can later be recognized at a different position, making 1D convnets
translation invariant (for temporal translations). 
- For instance, a 1D convnet processing
sequences of characters using convolution windows of size 5 should be able to
learn words or word fragments of length 5 or less

- The 2D poolingm operation has a 1D equivalent: extracting 1D patches (subsequences) from an input
and outputting the maximum value (max pooling) or average value (average pooling).
- Just as with 2D convnets, this is used for reducing the length of 1D inputs (subsampling).

## Working with IMDB

In [2]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras import layers
from keras import optimizers

In [3]:
max_features = 1000       # Consider only top 1000 words as features
max_len = 500             # Maximum length of review 500

In [4]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

In [5]:
print("Number of reviews in train = %d" %(len(x_train)))
print("Number of reviews in test = %d" %(len(x_test)))

Number of reviews in train = 25000
Number of reviews in test = 25000


In [12]:
x_padded_train = sequence.pad_sequences(x_train, maxlen = max_len)
x_padded_test = sequence.pad_sequences(x_test, maxlen= max_len)

## Tips for using 1D Convonets

- 1D convnets are structured in the same way as their 2D counterparts, which you used
in chapter 5: they consist of a stack of Conv1D and MaxPooling1D layers, ending in
either a global pooling layer or a Flatten layer, that turn the 3D outputs into 2D outputs,
allowing you to add one or more Dense layers to the model for classification or
regression.
- One difference, though, is the fact that you can afford to use larger convolution
windows with 1D convnets. 
- With a 2D convolution layer, a 3 × 3 convolution window
contains 3 × 3 = 9 feature vectors; but with a 1D convolution layer, a convolution window
of size 3 contains only 3 feature vectors. 
- You can thus easily afford 1D convolution windows of size 7 or 9.

In [8]:
model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len))
model.add(layers.Conv1D(32, kernel_size=7, activation='relu'))
model.add(layers.MaxPool1D(5))
model.add(layers.Conv1D(32, kernel_size=7, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(1))


In [10]:
opt = optimizers.Adam(lr = 0.001, decay=1e-5)
model.compile(optimizer =  opt, loss = 'binary_crossentropy', metrics=['acc'])

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 128)          128000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 163,937
Trainable params: 163,937
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = model.fit(x_padded_train, y_train, validation_data=(x_padded_test, y_test) ,epochs = 2, batch_size=32, verbose = 1)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 130s 5ms/step - loss: 0.6560 - acc: 0.6531 - val_loss: 0.5343 - val_acc: 0.7648
Epoch 2/2
25000/25000 [==============================] - 133s 5ms/step - loss: 0.4605 - acc: 0.8052 - val_loss: 0.4623 - val_acc: 0.8074
